In [2]:
import numpy as np
import sys
import dimod
import matplotlib.pyplot as plt
import os
import re

from dwave.system import EmbeddingComposite, DWaveSampler
#from pythia_generator import generator
from qubo_generator import make_qubo
from embedding_generator import generate_embedding

In [2]:
num_reads = 1000

In [1]:
# Events can be generated either through Pythia's python wrapper numpythia or by importing events generated with Pythia8:

particles = generator() #Generate particles with numpythia

In [17]:
# Use particles in event to generate qubo
qubo, offset = make_qubo(particles, np.pi/4.)
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo)

In [18]:
# Submit qubo to D-Wave

sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
#response = sampler.sample(bqm, num_reads=num_reads)
response = sampler.sample_qubo(qubo, num_reads=1000,return_embedding=True)

In [19]:
# Store energy and solutions of first run
qpu_energies = []
qpu_solutions = []

for datum in response.data(['sample', 'energy', 'chain_break_fraction']):
    qpu_energies.append(datum.energy)
    qpu_solutions.append(datum.sample)
    #print(datum.sample)

In [20]:
with open('out.particles', 'w') as f:
    for p in particles:
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %( p['px'], p['py'], p['pz'], p['E']))
        
with open('out.particlesqujet', 'w') as f:
    for i in range(len(particles)):
        key = 's[%s]' %i
        if(qpu_solutions[0][key]==1):
            f.write("0 0 0 0\n")
            f.write("%s %s %s %s\n\n\n" %( particles[i]['px'], particles[i]['py'], particles[i]['pz'], particles[i]['E']))

In [ ]:
# Loop over 100 lowest solutions to check which variables are always included in lowest enhergy solutions
freq = [0]*len(particles)

for i in range(0,1000):
    for p in range(len(particles)):
        key = 's[%s]' %p
        if(qpu_solutions[i][key] == 1): freq[p] +=1

In [ ]:
response_bqm = sampler.sample(bqm, num_reads=1000)

In [ ]:
response_bqm

In [ ]:
#Attempt to fix variables

for i in range(len(particles)):
    if(freq[i]/num_reads < 0.65): bqm

In [ ]:
response_adj = sampler.sample_qubo(qubo, num_reads=10000,return_embedding=True, chain_strength = 3000)

In [ ]:
qpu_energies_adj = []
qpu_chb_fraction_adj = []

for datum in response_adj.data(['sample', 'energy', 'chain_break_fraction']):
    qpu_energies_adj.append(datum.energy)
    qpu_chb_fraction_adj.append(datum.chain_break_fraction)

In [ ]:
from dwave.system import VirtualGraphComposite

qpu = DWaveSampler(solver='Advantage_system1.1')

In [ ]:
qpu.properties['extended_j_range']

In [ ]:
embedding = generate_embedding(len(particles))

virt_sampler = VirtualGraphComposite(qpu, embedding, chain_strength=1) #doctest: +SKIP